In [1]:
import os
os.getcwd()

'/root'

In [2]:
# !unzip Match_Data.zip

In [3]:
import gc

In [4]:
import pickle
import os
Name = ['MeltedToast', 'Mithraaaa']
feat = []
army = []
Y = []
id = []
for name in Name:
    file_path = '/root/Match_Data/'+name
    replays = os.listdir(file_path)
    for item in replays:
        replay_path = file_path + '/' + item
        with open(replay_path,"rb") as file:
            total = pickle.load(file)
            feat.append(total["features"])
            army.append(total["armies"])
            Y.append(total["kings"])
            id.append(total["id"])

# print(len(X))
# print(data[0])

In [5]:
mx = 0
feat_new = []
army_new = []
Y_new = []
for item in range(0,10374):
    mx = max(mx, len(Y[item]))
    if (len(Y[item])<=1000):
        feat_new.append(feat[item])
        army_new.append(army[item])
        Y_new.append(Y[item])
        # print(id[item])
print(mx)
del feat, army, Y, id
gc.collect()
import random
random_integers = random.sample(range(0, 10001), 3000)
# print(random_integers)
feat = [feat_new[item]for item in random_integers]
army = [army_new[item]for item in random_integers]
Y = [Y_new[item]for item in random_integers]

2099


In [6]:
del feat_new, army_new, Y_new
gc.collect()

0

In [7]:
print(Y[0])

[(0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), (0, 10), 

In [8]:
import numpy as np
print(len(feat))
def transform_to_numpy(X):
    res = []
    for item in X:
        XX = []
        flag=0
        for item2 in item:
            try:
                # print(item2)
                # print("TRY1: ",get_list_depth(item2),len(item2),)
                XX.append(np.array(item2))
            except ValueError as e:
                print(item2)
                # print("TRY2: ",get_list_depth(item2),len(item2))
                flag=1
                break
        if flag:
            break
        res.append(np.array(XX))
    return res

feat_np = transform_to_numpy(feat)
army_np = transform_to_numpy(army)
Y_np = []
for item in Y:
    Y_np.append(np.array(item))
# print(Y_np[0])

3000


In [12]:
del feat
del army
del Y
gc.collect()

1156

In [13]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
feat_flatten = [[step.flatten() for step in sequence] for sequence in feat_np]
feat_padded_flat = pad_sequences(feat_flatten, padding='post', value=-1.0)
feat_padded = np.array([np.array([step.reshape(25, 25) for step in sequence]) for sequence in feat_padded_flat])

army_flatten = [[step.flatten() for step in sequence] for sequence in army_np]
army_padded_flat = pad_sequences(army_flatten, padding='post', value=-1.0)
army_padded = np.array([np.array([step.reshape(25, 25) for step in sequence]) for sequence in army_padded_flat])

Y_flatten = [[step.flatten() for step in sequence] for sequence in Y_np]
Y_padded_flat = pad_sequences(Y_flatten, padding='post', value=-1.0)
Y_padded = np.array([np.array([step.reshape(1, 2) for step in sequence]) for sequence in Y_padded_flat])

2024-09-20 14:13:55.445373: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [14]:
del feat_np,army_np,Y_np
gc.collect()

0

In [16]:
print(feat_padded.shape)
print(army_padded.shape)
print(Y_padded.shape)

(3000, 992, 25, 25)
(3000, 992, 25, 25)
(3000, 992, 1, 2)


In [17]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Flatten, LSTM, Dense, TimeDistributed, Concatenate, Reshape, Activation
from keras import backend as K

# Input
input_1 = Input(shape=(None, 25, 25, 1))
input_2 = Input(shape=(None, 25, 25, 1))

# Conv2D & Flatten
conv1_1 = TimeDistributed(Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))(input_1)
conv1_1 = TimeDistributed(Flatten())(conv1_1)

conv1_2 = TimeDistributed(Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))(input_2)
conv1_2 = TimeDistributed(Flatten())(conv1_2)

merged = Concatenate(axis=-1)([conv1_1, conv1_2])

# LSTM Layer
lstm_out = LSTM(units=64, return_sequences=True, recurrent_activation='sigmoid')(merged)

# 50 Units: 2 Classification, Each 25 options
dense_out = Dense(units=50)(lstm_out)

# Shape: (batch_size, time_steps, 2, 25)
reshaped_out = Reshape((-1, 2, 25))(dense_out)

output = Activation('softmax')(reshaped_out)

model = Model(inputs=[input_1, input_2], outputs=output)

def loss_func(y_true, y_pred):
    ignore_steps = 20
    y_true = tf.cast(y_true, tf.float32)
    y_pred_labels = tf.cast(tf.argmax(y_pred, axis=-1), tf.float32)
    print(y_pred_labels.shape, y_true.shape)
    time_steps = tf.shape(y_true)[1]
    
    mask = tf.reduce_all(tf.equal(y_true, -1), axis=-1)
    mask = tf.cast(mask, dtype=tf.float32)
    
    loss = tf.abs(y_pred_labels[:, ignore_steps:, :] - y_true[:, ignore_steps:, :])
    loss = loss * (1 - mask[:, ignore_steps:])[:, :, None]
    return tf.reduce_mean(loss)

model.compile(optimizer='adam', loss="categorical_crossentropy")

model.summary()


2024-09-20 14:14:26.458411: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-20 14:14:26.896289: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22134 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:2e:00.0, compute capability: 8.9


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, 25, 2  0           []                               
                                5, 1)]                                                            
                                                                                                  
 input_2 (InputLayer)           [(None, None, 25, 2  0           []                               
                                5, 1)]                                                            
                                                                                                  
 time_distributed (TimeDistribu  (None, None, 23, 23  320        ['input_1[0][0]']                
 ted)                           , 32)                                                         

In [18]:
!pip install scikit-learn

Looking in indexes: http://mirrors.aliyun.com/pypi/simple


In [19]:
from sklearn.model_selection import train_test_split

X1_train, X1_test, X2_train, X2_test, y_train, y_test = train_test_split(
    feat_padded, army_padded, Y_padded, test_size=0.2, random_state=42
)

In [20]:
del feat_padded, army_padded, Y_padded
gc.collect()

3144

In [21]:
print(y_train.shape)

(2400, 992, 1, 2)


In [22]:
y_train = np.squeeze(y_train, axis=2)
y_test = np.squeeze(y_test, axis=2)

In [23]:
num_classes = 25

y_train_class_1 = tf.one_hot(y_train[..., 0], depth=num_classes)
y_train_class_2 = tf.one_hot(y_train[..., 1], depth=num_classes)
y_train_categorical = tf.stack([y_train_class_1, y_train_class_2], axis=-2)

y_test_class_1 = tf.one_hot(y_test[..., 0], depth=num_classes)
y_test_class_2 = tf.one_hot(y_test[..., 1], depth=num_classes)
y_test_categorical = tf.stack([y_test_class_1, y_test_class_2], axis=-2)

In [24]:
print(y_train_categorical.shape)
print(y_test_categorical.shape)

(2400, 992, 2, 25)
(600, 992, 2, 25)


In [25]:
print(X1_train.shape)
print(X2_train.shape)
print(y_train.shape)

(2400, 992, 25, 25)
(2400, 992, 25, 25)
(2400, 992, 2)


In [27]:
model.fit([X1_train, X2_train], y_train_categorical, epochs=100, batch_size=16, verbose=1)
model.save('model_full.h5')
model.evaluate([X1_test, X2_test], y_test_categorical, verbose=1)

Epoch 1/100
150/150 [==============================] - 27s 142ms/step - loss: 0.5835
Epoch 2/100
150/150 [==============================] - 22s 148ms/step - loss: 0.5295
Epoch 3/100
150/150 [==============================] - 21s 143ms/step - loss: 0.4822
Epoch 4/100
150/150 [==============================] - 20s 131ms/step - loss: 0.4367
Epoch 5/100
150/150 [==============================] - 20s 133ms/step - loss: 0.4046
Epoch 6/100
150/150 [==============================] - 22s 146ms/step - loss: 0.3840
Epoch 7/100
150/150 [==============================] - 18s 120ms/step - loss: 0.3439
Epoch 8/100
150/150 [==============================] - 21s 140ms/step - loss: 0.3107
Epoch 9/100
150/150 [==============================] - 20s 133ms/step - loss: 0.2843
Epoch 10/100
150/150 [==============================] - 19s 130ms/step - loss: 0.2622
Epoch 11/100
150/150 [==============================] - 20s 136ms/step - loss: 0.2448
Epoch 12/100
150/150 [==============================] - 19s 126

1.1553493738174438